## Descripcion del Problema

El problema consiste en predecir si un usuario de una aplicacion de audiolibros, es probable que vuelva a comprar un libro o no, para de esto modo, apuntar la publicidad a aquellos que seguramente vuelvan a comprar.

Para eso, el dataset consiste en una serie de datos de comportamiento de los usuarios, recopilados mediante la aplicacion durante 2 años. Luego, la variable dependiente o el target, es un valor entre 0 y 1 que indica si en los 6 meses siguientes, el usuario compro otro libro o no, ya que aquellos que luego de 2 años compraron otro libro, muy probablemente lo vuelvan a hacer, mientras que el resto, si durante esos 6 meses no compraron nada, es muy probable que no esten usando mas la aplicacion

## Importando el Dataset

In [2]:
import numpy as np
from sklearn import preprocessing

In [3]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

unscaled_inputs = raw_csv_data[:, 1:-1]
targets = raw_csv_data[:, -1]

## Balanceando el Dataset

Balancear el dataset consiste en igualar el numero de muestras para cada uno de los valores de salida, ya que si por ejemplo tenemos 2 posibles valores de salida pero hay mas muestras con un valor de salida que con el otro, el primero tendra mas peso por asi decirlo, por lo tanto el modelo se dara cuenta que la mayor cantidad de veces, las muestras corresponden a ese valor y comenzara a retornarlo mas.

In [4]:
num_one_targets = int(np.sum(targets))
zero_targets_count = 0
indices_to_remove = []

for i in range(targets.shape[0]):
    if zero_targets_count < num_one_targets:
        if targets[i] == 0:
            zero_targets_count += 1
            indices_to_remove.append(i)
    else:
        break
        
unscaled_inputs_balanced = np.delete(unscaled_inputs, indices_to_remove, axis=0)
targets_balanced = np.delete(targets, indices_to_remove, axis=0)

## Satandardizing

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_balanced)

## Shuffling

Se mezclan los datos para que en caso de que estuvieran ordenados por algun parametro o por la salida, se rompa ese orden y el batching funcione bien.

In [11]:
# se crea un array con indices de 0 a la cantidad de muestras del dataset y se mezcla
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# se generan los datasets mezclados usando como indices los creados anteriormente
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_balanced[shuffled_indices]

## Separando el dataset en Training, Validation y Test

In [12]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

In [13]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## Cargando los Datos

In [3]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

## Creando el Modelo

In [5]:
import tensorflow as tf

In [14]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
batch_size = 100
max_epochs = 100

# este es un callback que cortara el proceso de entrenamiento cuando se detecte que la validation loss aumenta respecto
# del epoch anterior (o de n epochs anteriores, definidos con el parametro patience), para prevenir overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_inputs, 
    train_targets, 
    batch_size=batch_size, 
    epochs=max_epochs, 
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets), 
    verbose=2
)

Train on 9477 samples, validate on 1184 samples
Epoch 1/100
9477/9477 - 1s - loss: 0.4492 - accuracy: 0.8087 - val_loss: 0.3302 - val_accuracy: 0.8809
Epoch 2/100
9477/9477 - 0s - loss: 0.2954 - accuracy: 0.8925 - val_loss: 0.2902 - val_accuracy: 0.8902
Epoch 3/100
9477/9477 - 0s - loss: 0.2697 - accuracy: 0.8987 - val_loss: 0.2681 - val_accuracy: 0.8919
Epoch 4/100
9477/9477 - 0s - loss: 0.2567 - accuracy: 0.9030 - val_loss: 0.2680 - val_accuracy: 0.8961
Epoch 5/100
9477/9477 - 0s - loss: 0.2528 - accuracy: 0.9027 - val_loss: 0.2590 - val_accuracy: 0.8944
Epoch 6/100
9477/9477 - 0s - loss: 0.2464 - accuracy: 0.9044 - val_loss: 0.2540 - val_accuracy: 0.9037
Epoch 7/100
9477/9477 - 0s - loss: 0.2422 - accuracy: 0.9060 - val_loss: 0.2500 - val_accuracy: 0.8978
Epoch 8/100
9477/9477 - 0s - loss: 0.2399 - accuracy: 0.9072 - val_loss: 0.2389 - val_accuracy: 0.9012
Epoch 9/100
9477/9477 - 0s - loss: 0.2370 - accuracy: 0.9074 - val_loss: 0.2401 - val_accuracy: 0.9029
Epoch 10/100
9477/9477 - 

## Evaluando el Modelo

In [17]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1186/1186 [==============================] - 0s 37us/sample - loss: 0.2429 - accuracy: 0.9030


In [18]:
print('Test loss: {0: .2f}. - Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100.))

Test loss:  0.24. - Test accuracy:  90.30%
